<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Introduction to Programming 2477-2223 S1
## Nova School of Business and Economics


### Team: Daniel Brosko (55036), ... (TODO)

TODO: theoretical description

## 1 Introduction

TODO

## 2 General packages loading

In [1]:
# in the beginning, we (install and) import all the needed packages

# !pip install pandas-datareader
# !pip install yfinance
# !pip install pyreadr
# !pip install fix-yahoo-finance==0.1.30


import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import yfinance as yf


#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp

In [66]:
pip list

Package              Version
-------------------- -----------
anyio                3.6.1
appdirs              1.4.4
appnope              0.1.3
argon2-cffi          21.3.0
argon2-cffi-bindings 21.2.0
asttokens            2.1.0
attrs                22.1.0
Babel                2.10.3
backcall             0.2.0
beautifulsoup4       4.11.1
bleach               5.0.1
bs4                  0.0.1
certifi              2022.9.24
cffi                 1.15.1
charset-normalizer   2.1.1
click                8.1.3
contourpy            1.0.5
cycler               0.11.0
debugpy              1.6.3
decorator            5.1.1
defusedxml           0.7.1
distlib              0.3.6
entrypoints          0.4
et-xmlfile           1.1.0
executing            1.2.0
fastjsonschema       2.16.2
filelock             3.8.0
Flask                2.2.2
fonttools            4.37.1
googlefinance        0.7
idna                 3.4
ipykernel            6.16.0
ipython              8.7.0
ipython-genutils     0.2.0
itsdangerous

In [2]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")

0     TER
1     ADM
2    TMUS
3    IDXX
4     LNC
Name: Ticker, dtype: object

## 3 Market Capitalization Data Scraping

Here we scrape the market capitalizations of the firms using own scraping function from scraping_function.py.
The data are in billions of USD.

In [3]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

#mkt_cap_df_list

Elapsed time: 1.453 seconds.


In [41]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks

df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names
df_mkt_cap.set_index('Date', inplace=True)

df_mkt_cap

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3486925294.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3486925294.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3486925294.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3486925294.py:6: 

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-04-01,0.78,0.00,6.24,2.04,1.84,1.07,31.68,4.98,4.81,12.40,27.33,13.47,9.12,1.17,14.97,6.26,94.12,6.70,1.93,7.62
2009-04-02,0.84,0.00,6.60,2.07,1.69,1.11,32.90,5.38,4.89,12.81,28.56,13.66,8.65,1.23,15.07,6.91,92.64,7.07,2.00,8.08
2009-04-03,0.89,0.00,6.45,2.13,1.79,1.10,33.69,5.52,4.96,12.64,29.27,13.57,8.76,1.27,14.52,7.01,91.16,7.25,2.07,8.12
2009-04-06,0.88,0.00,6.32,2.16,1.66,1.09,33.61,5.47,5.00,12.75,27.93,13.38,8.79,1.21,14.32,6.92,92.24,7.05,1.93,7.96
2009-04-07,0.83,0.00,5.95,2.14,1.79,1.06,32.54,5.41,4.86,12.58,26.43,13.31,8.50,1.17,15.03,6.61,90.90,6.72,1.80,7.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-28,14.11,54.00,187.96,33.31,6.37,16.68,957.44,19.86,53.96,62.69,65.00,23.04,51.19,4.16,42.15,64.76,277.94,40.81,24.53,12.21
2022-11-29,13.95,54.11,186.35,34.04,6.46,16.63,941.85,19.86,53.78,63.71,66.26,22.95,51.39,4.32,42.08,64.60,277.49,41.83,24.81,12.20
2022-11-30,14.61,54.70,189.91,35.45,6.61,17.46,983.84,20.23,54.72,64.81,67.45,23.65,51.86,4.47,44.07,65.12,281.08,42.30,25.06,12.53


## 4 Getting stock prices

In [6]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 21.494 seconds.


In [8]:
# loading stock prices into list of dataframes using own function
stocks_df_list = sp.load_stocks(success_tickers)
stocks_df_list

[            Date       High        Low       Open      Close     Volume  \
 0     2017-12-06  40.139999  39.110001  39.259998  39.959999  1201200.0   
 1     2017-12-07  40.660000  40.130001  40.270000  40.299999  1498300.0   
 2     2017-12-08  40.980000  40.220001  40.779999  40.509998  1779400.0   
 3     2017-12-11  40.919998  40.119999  40.650002  40.459999  1388200.0   
 4     2017-12-12  41.849998  40.610001  41.730000  40.950001  1837700.0   
 ...          ...        ...        ...        ...        ...        ...   
 1253  2022-11-29  90.870003  88.870003  90.139999  89.190002  1099900.0   
 1254  2022-11-30  93.480003  87.860001  89.669998  93.449997  2426400.0   
 1255  2022-12-01  94.610001  90.410004  94.370003  92.290001  1552900.0   
 1256  2022-12-02  92.809998  90.000000  90.110001  92.669998  1177200.0   
 1257  2022-12-05  93.800003  91.660004  92.150002  93.389999  1483173.0   
 
       Adj Close Name  
 0     38.815983  TER  
 1     39.146248  TER  
 2     39.3502

In [40]:
df_close_prices = stocks_df_list[0][['Date','Adj Close']]
 

for index in range(1, len(stocks_df_list)):
    dataframe = stocks_df_list[index][['Date','Adj Close']]
    df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')

cp_columns_names = ['Date']
cp_columns_names.extend(success_tickers)
df_close_prices.columns = cp_columns_names
df_close_prices.set_index('Date', inplace=True)

df_close_prices

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3683033131.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3683033131.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_49363/3683033131.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2017-12-06,38.815983,35.836540,62.119999,159.990005,66.111259,97.386032,57.617500,71.226997,245.830002,25.240000,46.197304,69.186127,698.650024,132.418320,314.660004,67.507912,28.125942,21.155281,25.719208,77.048676
2017-12-07,39.146248,35.914780,61.919998,159.779999,67.053230,97.798378,57.989498,71.545105,244.860001,25.040001,46.459652,69.277176,702.340027,132.865402,319.570007,68.817261,28.078480,21.214893,25.567820,77.287354
2017-12-08,39.350224,36.088657,62.540001,160.570007,67.874237,97.818008,58.099998,72.943306,251.070007,25.770000,46.848934,69.558586,721.890015,134.524612,325.769989,68.844009,28.268314,21.274504,25.542589,78.136086
2017-12-11,39.301666,36.427719,63.070000,159.699997,66.690269,97.278023,58.445999,71.596893,247.240005,25.500000,46.772770,70.005531,708.609985,133.690033,328.029999,69.271545,28.640057,21.535316,25.483717,77.773613
2017-12-12,39.777634,36.314709,63.480000,159.270004,67.035950,97.199478,58.254002,71.900215,246.860001,25.610001,46.967411,68.912979,711.049988,132.080536,327.750000,69.262627,28.932703,21.669445,25.609873,77.800133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-29,89.190002,96.449997,148.619995,408.959991,38.090000,223.500000,92.419998,139.369995,849.729980,44.500000,44.590000,112.809998,2556.050049,64.879997,291.410004,162.149994,49.490002,34.320000,36.000000,97.005951
2022-11-30,93.449997,97.500000,151.460007,425.869995,38.939999,234.660004,96.540001,141.949997,864.539978,45.270000,45.389999,116.269997,2579.000000,67.180000,305.170013,163.449997,50.130001,34.700001,36.360001,99.610001
2022-12-01,92.290001,91.320000,151.809998,439.170013,38.730000,242.020004,95.500000,142.490005,857.179993,46.119999,45.349998,116.300003,2552.969971,73.519997,301.850006,163.360001,51.080002,34.919998,35.770000,98.769997


In [44]:
# Making returns from daily prices
daily_returns = df_close_prices.pct_change(1) # 1 for ONE DAY lookback
daily_returns.dropna(inplace=True)
daily_returns.head()

# monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
# annual_return = prices.pct_change(252)

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2017-12-07,0.008508,0.002183,-0.003220,-0.001313,0.014248,0.004234,0.006456,0.004466,-0.003946,-0.007924,0.005679,0.001316,0.005282,0.003376,0.015604,0.019395,-0.001687,0.002818,-0.005886,0.003098
2017-12-08,0.005211,0.004841,0.010013,0.004944,0.012244,0.000201,0.001906,0.019543,0.025361,0.029153,0.008379,0.004062,0.027836,0.012488,0.019401,0.000389,0.006761,0.002810,-0.000987,0.010982
2017-12-11,-0.001234,0.009395,0.008475,-0.005418,-0.017444,-0.005520,0.005955,-0.018458,-0.015255,-0.010477,-0.001626,0.006425,-0.018396,-0.006204,0.006937,0.006210,0.013150,0.012259,-0.002305,-0.004639
2017-12-12,0.012111,-0.003102,0.006501,-0.002693,0.005183,-0.000807,-0.003285,0.004237,-0.001537,0.004314,0.004161,-0.015607,0.003443,-0.012039,-0.000854,-0.000129,0.010218,0.006228,0.004950,0.000341
2017-12-13,-0.002930,-0.041178,0.005514,-0.006718,-0.022174,-0.002626,-0.000815,-0.010392,-0.003808,0.003905,-0.016216,0.000721,-0.003994,-0.002482,0.009031,-0.002444,0.001640,-0.000688,-0.016749,-0.002614


In [ ]:
# continue work here

### 5 Market Data from prof. Kenneth French 

We dowloaded data about the market from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html , however, we need to change the date into suitable format.

In [61]:
data_market = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3)
data_market = data_market[:-1]

data_market.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.strptime(x, '%Y%m%d')
data_market['Date'] = data_market['Date'].map(date_format_lambda)

data_market.head()

,Date,Mkt-RF,SMB,HML,RF
0,1926-07-01,0.10,-0.25,-0.27,0.009
1,1926-07-02,0.45,-0.33,-0.06,0.009
2,1926-07-06,0.17,0.30,-0.39,0.009
3,1926-07-07,0.09,-0.58,0.02,0.009
4,1926-07-08,0.21,-0.38,0.19,0.009


So far just to put it together, we obtained the following dataframes we are going to use:


* df_mkt_cap - market capitalization
    
* daily_returns - daily stock returns

* data_market - FF market factors

In [ ]:
daily_returns.head()

### 6 Book to Value Ratio Scraping

In [ ]:
# TODO - download book to market ratio

In [ ]:
# TODO - download market indeces

In [ ]:
# TODO - perform regression

In [ ]:
# TODO - put results into table

In [ ]:
# TODO - comment on findings

In [ ]:
# todo - solve issues

tickers = pd.read_csv("tickers_set.csv", sep = ",")
tickers = pd.DataFrame(tickers)
tickers.columns = ["Index", "Ticker"]
# tickers.head()

tickers = tickers.Ticker
# tickers.head()

long_names = pd.DataFrame(index = range(len(tickers)), columns = 2)

for i in range(len(tickers)):
    long_names[i,0] = tickers[i]
    header = str(curlGetHeaders("https://www.macrotrends.net/stocks/charts/" + tickers[i])[5])
    long_names[i,1] = "Location: https://www.macrotrends.net/stocks/charts/" + tickers[i] + "/" + header
    long_names[i,1] = gsub("/\r\n","",long_names[i,2])
    
    
long_names

#stocks <- pd.Data.Frame()

In [ ]:
# url = https://www.macrotrends.net/stocks/charts/AAPL/apple/price-book
#page = requests.get('https://www.macrotrends.net/stocks/charts/AAPL')#/apple/price-book')


In [ ]:
get.packages('rvest')
library(rvest)

stocks <- pd.Data.Frame(read.csv("tickers_set.csv", sep=","))
colnames(stocks) <- c("Index","Ticker") #maybe pridat adresu
head(stocks,3)

stock_names <- rep(NA,length(stocks$Ticker))
for (i in seq(1:length(stocks$Ticker))){
  stock_names[i] <- as.character(stocks[i,2])
}

long_names <- data.frame(matrix(NA, nrow = length(stocks$Ticker), ncol = 2))
for (i in seq(1:length(stocks$Ticker))){
  long_names[i,1] <- stock_names[i]
  long_names[i,2] <- gsub(paste0("Location: https://www.macrotrends.net/stocks/charts/", stock_names[i],"/"),"",as.character(curlGetHeaders(paste0("https://www.macrotrends.net/stocks/charts/",stock_names[i]))[5]))
  long_names[i,2] <- gsub("/\r\n","",long_names[i,2])
}
names(long_names) <- c("Ticker", "Long_Name")
print(long_names)
write.csv(long_names,"long_names.csv")
